<a href="https://colab.research.google.com/github/lbdlab/NGS-LBD/blob/master/notebooks/LBD_Bilirubin_Modulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LBD - Bilirubin Modulation**

In [ ]:
!pip install neo4j pandas seaborn matplotlib biopython py2neo

     |████████████████████████████████| 204kB 25.7MB/s 
     |████████████████████████████████| 368kB 38.6MB/s 
     |████████████████████████████████| 3.2MB 30.9MB/s 
     |████████████████████████████████| 153kB 47.4MB/s 
     |████████████████████████████████| 71kB 8.9MB/s 
  Created wheel for neotime: filename=neotime-1.7.4-cp37-none-any.whl size=20564 sha256=d6578b827d2abe5fec46a846a3320e463af928fc6e3bbe9de625a5df7c64247b
  Stored in directory: /root/.cache/pip/wheels/d4/b4/c4/ab867cd0ac9e667f8cd269f9364fd1cb8ee75fe50f5ce97c79
Successfully built neotime
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.19 which is incompatible.
ERROR: ipython 5.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.4, but you'll have prompt-toolkit 3.0.19 which is incompatible.
  Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18


In [ ]:
# Libraries
%matplotlib inline
from datetime import datetime
import pandas as pd
# from neo4j import GraphDatabase
from py2neo import Graph



In [ ]:
# Search in PubMed

from Bio import Entrez

Entrez.email = 'ziberna@gmail.com'


In [ ]:
# # Connect to Neo4j
# database_name = 'semmed4321'
# driver = GraphDatabase.driver('neo4j://212.235.239.171:7687', auth=('neo4j', 'semMedDb2020'))

# def read_query(query, params={}):
#     with driver.session(database=database_name) as session:
#         result = session.run(query, params)
#         return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [ ]:
# Custom functions

# Connect to graph function
def get_graph_instance():
    return Graph("bolt://212.235.239.171:7687", user='neo4j', password='semMedDb2020', name='semmed4321')

# Create PubMed link
def generate_pubmed_url(*, pmid: int) -> str:
    """Generate PubMed URL from PMID.
    
    :param pmid: PubMed ID number of an article
    :return: URL string
    """
    pubmed_stem_url = 'https://pubmed.ncbi.nlm.nih.gov/'
    pubmed_url = pubmed_stem_url + str(pmid)
    return pubmed_url

# Create dataframe and html file from output
def create_df_html_from_query_output(*, output: list, html_file_name: str) -> pd.DataFrame:
    """
    Create dataframe and save html file from Neo4J query results data.
    
    :param output: data output of a specific Neo4J query
    :param html_file_name: filename of the saved html file
    :return: dataframe of the output
    """
    # Create dataframe from output
    df = pd.DataFrame(output)

    # Search for a specific PMID
    all_pmid_list = df['c_xy.pmid'].to_list()

    handle = Entrez.efetch(db='pubmed', id=all_pmid_list, retmode="xml")
    record = Entrez.read(handle)

    # Extract Journal data
    pmid_dict = {}

    for article in record['PubmedArticle']:
        # Extract data
        try:
            pmid = str(article['MedlineCitation']['PMID'])
            journal_title = article['MedlineCitation']['Article']['Journal']['Title']
            journal_title_abbreviation = article['MedlineCitation']['Article']['Journal']['ISOAbbreviation']
            journal_issn = str(article['MedlineCitation']['Article']['Journal']['ISSN'])
        except:
            print('Not found PubMed data for PMID: ' + str(article['MedlineCitation']['PMID']))
            pmid = journal_title = journal_title_abbreviation = journal_issn = 'Not found'

        # Add to dict
        pmid_dict[pmid] = {'pmid': pmid,
                           'journal_title': journal_title,
                           'journal_title_abbreviation': journal_title_abbreviation,
                           'journal_issn': journal_issn}

    # Add journal data to dataframe
    df['journal_title'] = df['c_xy.pmid'].apply(
        lambda x: add_values_from_dict(input_dict=pmid_dict, key1=x, key2='journal_title'))
    df['journal_issn'] = df['c_xy.pmid'].apply(
        lambda x: add_values_from_dict(input_dict=pmid_dict, key1=x, key2='journal_issn'))
    

    # Add PubMed URL
    df['PubMed_URL'] = df['c_xy.pmid'].apply(lambda x: generate_pubmed_url(pmid=x))
    
    # Save dataframe to html
    with open(html_file_name, 'w') as html_file:
        html_string = df.to_html(render_links=True, notebook=False, show_dimensions=True, justify='center')
        html_file.write(html_string)
    
    # Return generated dataframe
    return df
    

def add_values_from_dict(*, input_dict: dict, key1: str, key2: str) -> str:
    """
    Extract data from custom two level dictionary.
    """
    try:
        out = input_dict[key1][key2]
    except Exception as e:
        print('Error in extracting data from dict.')
        print(repr(e))
        out = 'Not found'
    
    return out





---
## Heme Oxygenase-1

### Different names

Heme Oxygenase-1 (C0538674)
HMOX1 protein, human (C1565861)

### Partially connected
Heme Oxygenase (Decyclizing) (C0018969)
Hmox1 protein, mouse (C1565862)
Hmox1 protein, rat (C1453914)

heme oxygenase-2 (C0537535)
heme oxygenase-3 protein, human (C1435054)

### SemBT relations
http://sembt.mf.uni-lj.si/user_guide/SemBT_relation_types_and_instances_counts.html

AFFECTS	1008068	2124063
INTERACTS_WITH	956926	1824826

ASSOCIATED_WITH	544318	1316494
STIMULATES	442904	845725
INHIBITS	424125	749490
AUGMENTS




In [ ]:
query = """
MATCH (x:Concept)-[r_xy]-(y:Concept)
WHERE 
(x:phsu OR x:aapp OR x:chem OR x:orch OR x:inch OR x:sbst OR x:clnd) AND
// (x:phsu OR x:aapp) AND
(r_xy:AFFECTS OR r_xy:NEG_AFFECTS OR 
r_xy:ASSOCIATED_WITH OR r_xy:NEG_ASSOCIATED_WITH OR 
r_xy:STIMULATES OR r_xy:NEG_STIMULATES OR 
r_xy:INHIBITS OR r_xy:NEG_INHIBITS OR 
r_xy:INTERACTS_WITH OR r_xy:NEG_INTERACTS_WITH OR 
r_xy:AUGMENTS OR r_xy:NEG_AUGMENTS) AND
y.cui in ['C0538674', 'C1565861']

WITH x, r_xy, y ORDER BY r_xy.freq DESC LIMIT 100

CALL {
    WITH x, r_xy, y
    MATCH (c_xy)<-[r_in_xy:IS_IN]-(s_xy)<-[r_extr_xy:Extracted_From]-(i_xy:Instance {predicate: type(r_xy)})-[r_sub_x:Inst_Subject]->(x),
      (i_xy)-[r_obj_y:Inst_Object]->(y)
    RETURN c_xy, s_xy, i_xy LIMIT 10
}

RETURN DISTINCT x.name,
type(r_xy), 
y.name,
r_xy.freq, r_xy.min_pyear,
c_xy.pmid, c_xy.pyear, c_xy.issn,
i_xy.predicate, i_xy.indicator_type,
s_xy.normalized_section_header, s_xy.sentence

"""

In [ ]:
# Query
start_time = datetime.now()

# Run query
graph = get_graph_instance()
output = graph.run(query).data()

end_time = datetime.now()

# Time needed
print('Time needed: ' + str(end_time - start_time))

# Output size
print('Output rows: ' + str(len(output)))


In [ ]:
# Create dataframe and HTML file
hox1_df = create_df_html_from_query_output(output=output, html_file_name='outputs/heme_oxygenase-1.html')


In [ ]:
hox1_df

In [ ]:
# Unique compunds
hox1_df['x.name'].value_counts().iloc[0:10]
